# Подготовка данных



In [1]:
%enable_full_walk
import sys
import numpy as np
import pandas as pd

import os
import functools as ft
root = ft.reduce(os.path.join, ['..'])

sys.path.append(root)

import pickle
import scipy
from scipy.sparse import coo_matrix

from sklearn.model_selection import train_test_split

def load_mapping(filename):
    map_path = ft.reduce(os.path.join, ['..','data','mapping' ,filename+'.pkl'])
    with open(map_path, 'rb') as handle:
        return pickle.load(handle)

### Загрузка данных

In [14]:
path = ft.reduce(os.path.join, ['..','data', 'raw', 'purchases.csv'])
df = pd.read_csv(path, dtype={'transaction_id': 'str',
                                'product_id': 'str',
                                'store_id': 'str',
                                'client_id': 'str'}, #nrows=10, 
                 usecols=['client_id', 'transaction_id', 'transaction_datetime', 'store_id', 'product_id'])

df = df[df['product_id'].isin(df['product_id'].value_counts()[:200].index.tolist())]

# Убираем коллизии в индексах транзакций
transactions = df[['transaction_id', 'transaction_datetime', 'store_id']].drop_duplicates()
trans_without_collision = transactions.groupby('transaction_id').count()['transaction_datetime']
trans_without_collision = trans_without_collision[trans_without_collision == 1].index.values
del transactions

df = df[df.transaction_id.isin(trans_without_collision)]
del trans_without_collision

# Убираем корзины с малым количеством товаров
v_c = df['transaction_id'].value_counts() 
v_c = v_c[v_c > 2]
df = df[df.transaction_id.isin(v_c.index.values)]
del v_c

# Сортируем по времени
df.sort_values('transaction_datetime', inplace=True)
df.reset_index(inplace=True, drop=True)


### Создание словарей с внутренней индексацией

In [16]:
# Клиент
client_idx_to_id = dict(enumerate(df.client_id.unique()))
client_id_to_idx = {v: k for k, v in client_idx_to_id.items()}
# Продукт
product_idx_to_id = dict(enumerate(df.product_id.unique()))
product_id_to_idx = {v: k for k, v in product_idx_to_id.items()}
# Магазин
store_idx_to_id = dict(enumerate(df.store_id.unique()))
store_id_to_idx = {v: k for k, v in store_idx_to_id.items()}
# Транзакции
trans_idx_to_id = dict(enumerate(df.transaction_id.unique()))
trans_id_to_idx = {v: k for k, v in trans_idx_to_id.items()}

In [29]:
def save_mapping(structure, filename):
    map_path = ft.reduce(os.path.join, ['..','data','mapping' ,filename+'.pkl'])
    with open(map_path, 'wb') as handle:
        pickle.dump(structure, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
save_mapping(client_idx_to_id, 'client_idx_to_id')
save_mapping(client_id_to_idx, 'client_id_to_idx')

save_mapping(product_idx_to_id, 'product_idx_to_id')
save_mapping(product_id_to_idx, 'product_id_to_idx')

save_mapping(store_idx_to_id, 'store_idx_to_id')
save_mapping(store_id_to_idx, 'store_id_to_idx')

save_mapping(trans_idx_to_id, 'trans_idx_to_id')
save_mapping(trans_id_to_idx, 'trans_id_to_idx')


In [17]:
# Далее работаем с датасетом, где сохраняется только внутренняя индексация
df['transaction_idx'] = df.transaction_id.map(trans_id_to_idx).astype(int)
df['product_idx'] = df.product_id.map(product_id_to_idx).astype(int)
df['client_idx'] = df.client_id.map(client_id_to_idx).astype(int)
df['store_idx'] = df.store_id.map(store_id_to_idx).astype(int)

cols = ['transaction_datetime', 'transaction_idx', 'client_idx', 'store_idx', 'product_idx']

df = df[cols]
df.to_csv(ft.reduce(os.path.join, ['..', 'data', 'processed', 'purchases.csv']), index=False)

### Сбор контекста корзин

In [669]:
df = pd.read_csv(ft.reduce(os.path.join, ['..','data', 'processed', 'purchases.csv']), 
                 dtype={'transaction_idx': 'int',
                        'product_idx': 'int',
                        'store_idx': 'int',
                        'client_idx': 'int'})

In [19]:
# Берем уникальные транзакции
transactions = df[['transaction_idx', 'transaction_datetime', 'store_idx']].drop_duplicates()
transactions.set_index('transaction_idx', inplace=True)


# Для каждой корзины сохраняем количество товаров
transactions['product_count'] = df['transaction_idx'].value_counts()

# Генерируем временной контекст
transactions.transaction_datetime = transactions.transaction_datetime.apply(lambda x: pd.to_datetime(x))
transactions['hour'] = transactions.transaction_datetime.apply(lambda x: x.hour)
transactions['dayofweek'] = transactions.transaction_datetime.apply(lambda x: x.dayofweek)
transactions['day'] = transactions.transaction_datetime.apply(lambda x: x.day)
transactions['month'] = transactions.transaction_datetime.apply(lambda x: x.month)
transactions['year'] = transactions.transaction_datetime.apply(lambda x: x.year)

In [20]:
# Сохранение контекста корзин
trans_context_columns = ['store_idx', 'product_count', 'hour', 'dayofweek', 'day', 'month', 'year']
transactions = transactions[trans_context_columns]

transactions.to_csv(ft.reduce(os.path.join, ['..', 'data', 'interim', 'trans_context.csv']), index=True)

### Сбор профиля клиента
Клиент представлен в виде конкатенации двух векторов - истории покупок и текущей корзины.

История покупок для неидентифицированных клиентов - нулевой вектор.

In [694]:
# Загрузка данных
df = pd.read_csv(ft.reduce(os.path.join, ['..','data', 'processed', 'purchases.csv']), 
                 dtype={'transaction_idx': 'int',
                        'product_idx': 'int',
                        'store_idx': 'int',
                        'client_idx': 'int'})

transactions = pd.read_csv(ft.reduce(os.path.join, ['..', 'data', 'interim', 'trans_context.csv']))
transactions.set_index('transaction_idx', inplace=True)

product_id_to_idx = load_mapping('product_id_to_idx')

In [21]:
# Последнюю транзакцию клиента будем использовать, как текущую корзину
client_basket = df.groupby('client_idx').apply(lambda x: x[x.transaction_datetime == x.transaction_datetime.max()])
client_history = df.groupby('client_idx').apply(lambda x: x[x.transaction_datetime != x.transaction_datetime.max()])

client_basket.reset_index(drop=True, inplace=True)
client_history.reset_index(drop=True, inplace=True)

products_list = list(product_id_to_idx.values())

def get_negative_sample(products_list, basket):
    """Получение негативного таргета для корзины клиента. 
    Негативный таргет - случайный товар из католога за исключением товаров в корзине."""
    
    neg_samples = list(set(products_list)-set(basket))  
    return np.random.choice(neg_samples, 1)[0]

# Создание отрицательного таргета для каждого клиента
neg_target = client_basket.groupby('client_idx').product_idx\
    .apply(lambda x: get_negative_sample(products_list, x.values))

# Выберем случайный товар для положительного таргета из корзины
target = client_basket.groupby('client_idx').product_idx.apply(lambda x: x.sample(n=1).values[0])
target = target.reset_index()

# Убираем таргет из корзины
without_target = pd.concat([client_basket[['client_idx', 'product_idx']], target, target]).drop_duplicates(keep=False)

# Не учитываем количество товаров в истории
client_history = client_history[['client_idx', 'product_idx']].drop_duplicates() 

In [22]:
# Делаем матричное представление для пар клиент-товар_в_истории
client_history['value'] = [True]*client_history.shape[0]
client_history = client_history.pivot(index='client_idx', columns='product_idx')['value'].fillna(False).add_prefix('h_')

# Делаем матричное представление для пар клиент-товар_в_корзине
without_target['value'] = [True]*without_target.shape[0]
without_target = without_target.pivot(index='client_idx', columns='product_idx')['value'].fillna(False).add_prefix('b_')

# Последние транзакции клиентов - текущая корзина
basket_transactions = client_basket[['client_idx', 'transaction_idx']].drop_duplicates()
basket_transactions = basket_transactions.set_index('client_idx', drop=True)

Зануляем историю для определенного % клиентов

In [23]:
# Реальные клиенты с только одной транзакцией
all_clients = df[['client_idx', 'transaction_idx']].drop_duplicates().groupby('client_idx').count()
new_clients = all_clients[all_clients < 2].dropna().index.values
old_clients = all_clients[all_clients > 1].dropna().index.values

ratio = 0.9
n = int(all_clients.shape[0]*ratio - len(new_clients)) # Кол-во клиентов для зануления истории

# Отбор кандидатов для зануления
candidates = np.random.choice(old_clients, n, replace=False)
client_history.loc[candidates] = False

In [24]:
# Контекст текущей корзины клиента
client_context = transactions.loc[basket_transactions.transaction_idx.values].set_index(basket_transactions.index)

# Сборка датасета с историей, текущей корзиной и контекстом покупки
client_full_df = client_history.join(without_target , how='right').fillna(False)
client_full_df = client_full_df.join(client_context)

# Флаг на нового клиента
client_full_df['is_new_client'] = ~client_full_df.loc[:, 'h_0':'h_199'].any(axis=1)

# Положительный таргет для клиентов
client_full_df_pos = client_full_df.copy()
client_full_df_pos['target_product'] = target.product_idx
client_full_df_pos['target'] = 1

# Отрицательный таргет для клиентов
client_full_df_neg = client_full_df.copy()
client_full_df_neg['target_product'] = neg_target
client_full_df_neg['target'] = 0

# Объединение + и - таргета в один датасет
client_full_df = pd.concat([client_full_df_pos, client_full_df_neg])
del client_full_df_neg, client_full_df_pos

# Сортировка клиентов по дате текущей корзины
client_full_df = client_full_df.sort_values(['year', 'month', 'day', 'hour', 'client_idx', 'target'])

# Запись
client_full_df.to_csv(ft.reduce(os.path.join, ['..', 'data', 'interim', 'client_hist_context_target.csv']), index=True)

#### Добавление атрибутов клиентов

In [25]:
client_full_df = pd.read_csv(ft.reduce(os.path.join, ['..', 'data', 'interim', 'client_hist_context_target.csv']))
client_full_df.set_index('client_idx', inplace=True)

clients = pd.read_csv(ft.reduce(os.path.join, ['..', 'data', 'raw', 'clients.csv']))

client_id_to_idx = load_mapping('client_id_to_idx')

In [26]:
# Работа с клиентским датасетом. Приведение индексов к внутреннему стандарту. 
clients['client_idx'] = clients.client_id.map(client_id_to_idx)
clients.dropna(inplace=True)
clients['client_idx'] = clients['client_idx'].astype(int)
clients.set_index('client_idx', inplace=True)

# Заполнение выбросов модой
mode = clients.age.mode()[0]
clients.age = clients.age.apply(lambda x: mode if x < 10 or x > 80 else x)

clients = clients[['age', 'gender']]

clients.to_csv(ft.reduce(os.path.join, ['..', 'data', 'processed', 'clients.csv']), index=True)

In [27]:
# Добавление доп информации о клиентах в основной датасет
client_full_df['age'] = clients['age']
client_full_df['gender'] = clients['gender']

client_full_df.gender.fillna('U', inplace=True)
client_full_df.age.fillna(mode, inplace=True)

#### Добавление атрибутов товара из таргета

In [28]:
# Загрузка данных о товарах и приведение индексов
products = pd.read_csv(ft.reduce(os.path.join, ['..', 'data', 'raw', 'products.csv']))
#product_id_to_idx = load_mapping('product_id_to_idx')
products['product_idx'] = products.product_id.astype('str').map(product_id_to_idx)

products.dropna(subset=['product_idx'], inplace=True)
products['product_idx'] = products['product_idx'].astype(int)

# Заполнение пропусков для категориальных признаков и label encoding
for column in ['level_1', 'level_2', 'level_3', 'level_4', 'segment_id', 'brand_id', 'vendor_id']:
    products[column].fillna('null', inplace=True)
    products[column] = products[column].astype('category').cat.codes
    
products.set_index('product_idx', inplace=True)
products = products[['level_1', 'level_2', 'level_3', 'level_4', 'segment_id', 'brand_id', 'vendor_id', 'is_own_trademark']]
products = products.add_prefix('prdt_')

# Запись информации о товарах
products.to_csv(ft.reduce(os.path.join, ['..', 'data', 'processed', 'products.csv']), index=True)

In [30]:
# Добавление доп информации о продуктах из таргета в основной датасет
client_full_df = client_full_df.join(products, on='target_product', how='left')

# label encoding
# 0-F, 1-M, 2-U
client_full_df['gender'] = client_full_df['gender'].astype('category').cat.codes

# Запись датасета для обучения и тестирования
client_full_df.to_csv(ft.reduce(os.path.join, ['..', 'data', 'processed', 'client_hist_context_target_product.csv']), index=True)